In [51]:
import matplotlib.pyplot as plt
import numpy as np

import camb

from scipy.optimize import curve_fit

In [110]:
# Load Data
pars     = np.load('CL_data/parameter_ombh_cross2.npy')  # [H0, ombh2, omch2 ] x 100
lensed   = np.load('CL_data/lensed_ombh_cross2.npy')     # [C_2, ..., C_5000] x 100 (lensed)
unlensed = np.load('CL_data/unlensed_ombh_cross2.npy')     # [C_2, ..., C_5000] x 100 (unlensed)

In [111]:
omcs = pars[:,1]
As = pars[:,3]
ombs = pars[:,2]

ells = np.array([l for l in range(2, 5000)])

In [112]:
#getting dependent variables
lensing = lensed/unlensed #lensing tail
camb_data_format = []
for i in range(len(lensing)):
    for j in range(len(lensing[i])):
        camb_data_format.append(lensing[i][j])

In [113]:
fid_ombh2 = 0.0224
fid_omch2 = 0.12

In [127]:
def func(X, b1):
    #expected form of the lensing tail function for curve fitting
    ell = X[0]
    x2 = (X[1]/0.12) - 1
    x1 = X[2]/ 0.0224 - 1
    x5 = X[3] / 3.043 - 1
        
    beta_1 =  0.6838  * (1 + 0.6785287  * x1 + 5.14232428 * x1 **2 +-0.0394 * x2 + 1.15286268 * x1 * x2 + 0.00295 * x5 + 0.0772 * x5**2 + 0.02446665 * x2 * x5)
    beta_2 = 2803.1562 * (1 + -0.20201989 * x1 + -0.1305 * x2 + -0.03669247 * x1 ** 2 + 0.1632 * x2 **2 + 0.01520631 * x1 * x2 +0.01288  * x5 + -0.1299 * x5**2 + -0.02780125 * x2 * x5)
    
    beta_3 = 3752.098 * (1 + 0.76363938 * x1 + -0.4728 * x2  + 0.3576 * x2**2  + 0.54528086 * x1**2 + 1 * x1 * x2 + -0.1017 * x5 + 0.1915  * x5**2 + 1.19211136 * x2 * x5 + -1.26835811 * x1 * x5)
    beta_4 = 407.009 * (1 + 1.47840849 * x1 + 0.22685781 * x1 **2 + -0.066 * x2 + -0.1976 *x2**2 + 1.0051622 * x1 * x2 + 0.5307 * x5 + 0.1483 * x5**2 + 1.016768 * x2 * x5 + 1.06461251 * x1 * x5)
    alpha = 2.5867 *(1 + -1.18769837 * x1 + 0.700  * x2 +0.27838511 * x1 **2  + 0.2064 * x2 **2 + -0.97586643 * x1 * x2 + 1.4088  * x5 + 0.7482    * x5**2 + 1.00601945 * x5 * x2 + -1.22167607 * x1 * x5)
        
    sigm = (1 + np.exp(-(ell-beta_3)/beta_4))**(-1)

    poly = (beta_1*(ell/beta_2)**alpha - 1)
    
    L = poly*sigm + 1
    
    return L

In [128]:
cosmos = [ombs[i] + omcs[i] for i in range(len(ombs))]    #x3 variable
n = len(cosmos) #number of cosmologies
params = np.zeros((len(ells) * n, 4))   #ind variables
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),0] = ells
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),1] = omcs[i]  #omch2
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),2] = ombs[i]  #ombh2
for i in range(n):
    params[(i) * len(ells): (i+1)*len(ells),3] = As[i]

In [129]:
answers, _ = curve_fit(func,(params[:,0], params[:,1], params[:,2], params[:,3]),camb_data_format)  #doing the curve fitting

/tmp/ipykernel_3609/3697458515.py:15: RuntimeWarning: overflow encountered in exp
  sigm = (1 + np.exp(-(ell-beta_3)/beta_4))**(-1)


In [130]:
answers

array([1.06461251])